# Глубинное обучение для текстовых данных, ФКН ВШЭ
## Домашнее задание 4: Direct Preference Optimization 

__Мягкий дедлайн 16.11.25 23:59__ \
__Жесткий дедлайн 19.11.25 23:59__

### О задании

В этом задании вам предстоит обучить большую LLM для ответов на вопросы с помощью DPO, а также реализовать LoRA для эффективного обучения. 

### Оценивание и штрафы

Максимально допустимая оценка за работу — __11 баллов__.

Оценка за это домашнее задание будет формироваться из оценки за __задания__ и за __отчет__, в котором от вас требуется написать о проделанной работе. За отчет можно получить до 2-х баллов, однако в случае отсутствия отчета баллы за соответствующие задания не будут ставиться. Мы настаиваем на том, чтобы вы оформили весь код в виде полноценного проекта. Этот ноутбук нужно рассматривать скорее как файл с условием, чем как место для написания массивного кода. За сдачу больших ноутбуков с кодом оценка будет снижена. Ответы на все вопросы в заданиях можно (нужно) писать в отчете.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

### План решения

<img src="https://miro.medium.com/v2/resize:fit:1400/1*lK6iJMz5CGh2fo7TsDn15A.png" alt="drawing" width="700"/>

Обучение следованию инструкциям с помощью DPO разбивается на два этапа:    
1. __Supervised Fine-tuning (SFT)__ – обучение базовой модели ответам на запросы в нужном формате.
2. __Direct Preference Optimization (DPO)__ – обучение SFT модели приоритизации "хороших" ответов.

Мы не хотим обучать модели целиком по двум причинам: 1) используемые модели очень большие; 2) нам требуется лишь выравнить модель с нашими предпочтениями, не внося в нее новых знаний, что не требует серьезного обучения. Поэтому мы будем использовать PEFT, а именно LoRA для обучения.

Таким образом, вам надо будет:
1. Реализовать и протестировать LoRA
2. Разобраться с данными и привести их к нужному формату
3. Обучить SFT модель
4. Обучить DPO модель
5. Порадоваться, что вы молодцы и со всем справились
6. (Опционально) сделать веб-интерфейс для вашей модели, переиспользуя код из первой домашки (мы можем выдать бонусы, если получится классно).

### О датасете

Мы будем работать с датасетом [Anthropic Helpful-Harmless](https://huggingface.co/datasets/Anthropic/hh-rlhf) для RLHF. В нем содержится 160к примеров ответов на вопросы с историей.

### Low-Rank Adaptation (LoRA)

<img src="https://heidloff.net/assets/img/2023/08/lora.png" alt="drawing" width="600"/>

__Задание 1 (3 балла).__ Реализуйте самостоятельно модуль LoRA для эффективного обучения LLM по схеме, описанной в [статье](https://arxiv.org/pdf/2106.09685). Встройте его в свою любимую LLM и убедитесь, что ошибка убывает при обучении параметров LoRA на безусловную генерацию. Для этого возьмите любые данные на свой выбор. Замерьте насколько уменьшилось число обучаемых параметров, как изменилась скорость во время forward и backward процессов и как изменились затраты по памяти. Сделайте выводы и напишите о них в отчете.

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling, TrainingArguments, Trainer

/home/sinfillo/DPO-NLP-HW/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class LoRA(nn.Module):
    def __init__(self, in_features, out_features, r, alpha, dropout, bias=True):
        super().__init__()

        self.in_features = in_features
        self.out_features = out_features
        self.r = r
        self.alpha = alpha
        self.scaling = alpha / r

        self.base = nn.Linear(in_features, out_features, bias=bias)

        self.lora_A = nn.Linear(in_features, r, bias=False)
        self.lora_B = nn.Linear(r, out_features, bias=False)

        self.dropout = nn.Dropout(dropout) if dropout > 0 else nn.Identity()

        self.base.reset_parameters()
        nn.init.normal_(self.lora_A.weight, mean=0.0, std=0.02)
        nn.init.zeros_(self.lora_B.weight)

    def forward(self, x):
        base_out = self.base(x)
        lora_out = self.lora_B(self.lora_A(self.dropout(x))) * self.scaling
        return base_out + lora_out


In [3]:
def freeze_base_and_unfreeze_lora(model):
    for p in model.parameters():
        p.requires_grad = False

    for name, p in model.named_parameters():
        if "lora_A" in name or "lora_B" in name:
            p.requires_grad = True

In [4]:
def count_parameters(model):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total, trainable

In [5]:
def get_parent_module(model, module_name):
    parts = module_name.split(".")
    parent = model
    for p in parts[:-1]:
        parent = getattr(parent, p)
    return parent, parts[-1]


In [ ]:
def apply_lora(model, r=8, alpha=16, dropout=0.05, target=("q_proj", "v_proj")):
    for name, module in list(model.named_modules()):
        if isinstance(module, nn.Linear) and any(t in name for t in target):
            parent, attr = get_parent_module(model, name)
            lora = LoRA(module.in_features, module.out_features, r=r, alpha=alpha, dropout=dropout, bias=(module.bias is not None))
            lora.base.weight.data = module.weight.data.clone()
            if module.bias is not None:
                lora.base.bias.data = module.bias.data.clone()
            setattr(parent, attr, lora)

In [7]:
from datasets import load_dataset

In [8]:
model_name = "Qwen/Qwen2-0.5B"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

raw_datasets = load_dataset("wikitext", "wikitext-2-raw-v1")

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=False, add_special_tokens=False)

tokenized = raw_datasets.map(tokenize_function, batched=True, remove_columns=["text"])

block_size = 128

def group_texts(examples):
    concatenated = sum(examples["input_ids"], [])
    total_length = (len(concatenated) // block_size) * block_size
    concatenated = concatenated[:total_length]
    result = {"input_ids": [concatenated[i:i+block_size] for i in range(0, total_length, block_size)]}
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized.map(group_texts, batched=True, remove_columns=tokenized["train"].column_names)

train_dataset = lm_datasets["train"]
eval_dataset = lm_datasets["validation"]

In [9]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [10]:
model_full = AutoModelForCausalLM.from_pretrained(model_name).to(device)
count_parameters(model_full)

(494032768, 494032768)

In [11]:
small_train = train_dataset.select(range(min(5000, len(train_dataset))))
small_eval = eval_dataset.select(range(min(1000, len(eval_dataset))))

In [12]:
training_args_full = TrainingArguments(
    output_dir="qwen_full_lm",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_ratio=0.03,
    logging_strategy="steps",
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    eval_steps=200
)

trainer_full = Trainer(
    model=model_full,
    args=training_args_full,
    train_dataset=small_train,
    eval_dataset=small_eval,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer_full.train()

/var/tmp/ipykernel_127079/2792372384.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_full = Trainer(
Detected kernel version 5.4.210, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Epoch,Training Loss,Validation Loss
1,2.964800,3.212937
2,1.748700,3.424326
3,0.694300,3.998857


TrainOutput(global_step=1875, training_loss=1.8451180308024089, metrics={'train_runtime': 1526.3335, 'train_samples_per_second': 9.827, 'train_steps_per_second': 1.228, 'total_flos': 4122986250240000.0, 'train_loss': 1.8451180308024089, 'epoch': 3.0})

In [13]:
model_lora = AutoModelForCausalLM.from_pretrained(model_name)

apply_lora(model_lora, r=8, alpha=16, dropout=0.05)
freeze_base_and_unfreeze_lora(model_lora)

model_lora.to(device)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): LoRA(
            (base): Linear(in_features=896, out_features=896, bias=True)
            (lora_A): Linear(in_features=896, out_features=8, bias=False)
            (lora_B): Linear(in_features=8, out_features=896, bias=False)
            (dropout): Dropout(p=0.05, inplace=False)
          )
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): LoRA(
            (base): Linear(in_features=896, out_features=128, bias=True)
            (lora_A): Linear(in_features=896, out_features=8, bias=False)
            (lora_B): Linear(in_features=8, out_features=128, bias=False)
            (dropout): Dropout(p=0.05, inplace=False)
          )
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
       

In [14]:
count_parameters(model_lora)

(494573440, 540672)

In [15]:
training_args_lora = TrainingArguments(
    output_dir="qwen_lora_lm",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=1e-3,
    weight_decay=0.01,
    warmup_ratio=0.03,
    logging_strategy="steps",
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch"
)

trainer_lora = Trainer(
    model=model_lora,
    args=training_args_lora,
    train_dataset=small_train,
    eval_dataset=small_eval,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer_lora.train()

/var/tmp/ipykernel_127079/1050235975.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_lora = Trainer(
Detected kernel version 5.4.210, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Epoch,Training Loss,Validation Loss
1,2.906000,2.841598
2,2.798700,2.845931
3,2.699500,2.861191


TrainOutput(global_step=1875, training_loss=2.833655810546875, metrics={'train_runtime': 1055.59, 'train_samples_per_second': 14.21, 'train_steps_per_second': 1.776, 'total_flos': 4129214791680000.0, 'train_loss': 2.833655810546875, 'epoch': 3.0})

In [16]:
from torch.utils.data import DataLoader

In [17]:
def short_loader(dataset, batch_size=8, max_batches=30):
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=data_collator)
    for i, batch in enumerate(loader):
        if i >= max_batches:
            break
        yield batch

In [18]:
import time
import torch

def measure_speed_and_memory(model, dataloader, device="cuda", lr=1e-4, max_steps=30):
    model.to(device)
    model.train()

    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)

    fwd_times = []
    bwd_times = []
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    steps = 0
    for batch in dataloader:
        x = batch["input_ids"].to(device)
        y = batch["labels"].to(device)

        torch.cuda.synchronize()
        t0 = time.perf_counter()

        out = model(input_ids=x, labels=y)
        loss = out.loss

        torch.cuda.synchronize()
        t1 = time.perf_counter()
        fwd_times.append(t1 - t0)




        optimizer.zero_grad(set_to_none=True)

        torch.cuda.synchronize()
        t2 = time.perf_counter()

        loss.backward()

        torch.cuda.synchronize()
        t3 = time.perf_counter()
        bwd_times.append(t3 - t2)

        optimizer.step()

        steps += 1
        if steps >= max_steps:
            break

    avg_fwd = sum(fwd_times) / len(fwd_times)
    avg_bwd = sum(bwd_times) / len(bwd_times)
    peak_mem = torch.cuda.max_memory_allocated() / 1024**2  # MB

    return {
        "avg_forward_time": avg_fwd,
        "avg_backward_time": avg_bwd,
        "peak_memory_mb": peak_mem,
        "steps": steps,
    }


In [19]:
full_stats = measure_speed_and_memory(model_full, short_loader(small_train), device="cuda", lr=1e-5, max_steps=30)
print("Qwen full stats:", full_stats)

Qwen full stats: {'avg_forward_time': 0.22811576579794443, 'avg_backward_time': 0.44181076536672964, 'peak_memory_mb': 18075.76220703125, 'steps': 30}


In [20]:
lora_stats = measure_speed_and_memory(model_lora, short_loader(small_train), device="cuda", lr=1e-3, max_steps=30)
print("LoRA stats:", lora_stats)

LoRA stats: {'avg_forward_time': 0.23334508403252888, 'avg_backward_time': 0.25204123323298216, 'peak_memory_mb': 14097.07080078125, 'steps': 30}


### Supervised Fine-tuning

__Задание 2 (3 балла).__ Разбейте все примеры с "хорошими" ответами на запросы (все что идет до последнего "Assistant:") и ответы (все, начиная с последнего "Assistant:"). Дообучите модель [`pythia-1.4b`](https://huggingface.co/EleutherAI/pythia-1.4b) генерировать правильные ответы с помощью вашей LoRA. Одной эпохи вполне должно хватить для сходимости. Проверьте на нескольких случайных тестовых примерах, что модель ведет себя так, как надо.

In [21]:
def apply_lora_to_pythia(model, r=8, alpha=16, dropout=0.05, target=("attention.query_key_value", "attention.dense", "mlp.dense_h_to_4h", "mlp.dense_4h_to_h")):
    for name, module in list(model.named_modules()):
        if isinstance(module, nn.Linear) and any(t in name for t in target):
            parent, attr = get_parent_module(model, name)
            lora = LoRA(module.in_features, module.out_features, r=r, alpha=alpha, dropout=dropout, bias=(module.bias is not None))
            lora.base.weight.data = module.weight.data.clone()
            if module.bias is not None:
                lora.base.bias.data = module.bias.data.clone()
            setattr(parent, attr, lora)

In [22]:
model_name = "EleutherAI/pythia-1.4b"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

dataset = load_dataset("Anthropic/hh-rlhf")
train_raw = dataset["train"]
test_raw = dataset["test"]

In [23]:
max_length = 256

def preprocess_sft(example):
    text = example["chosen"]
    marker = "Assistant:"
    idx = text.rfind(marker)

    if idx == -1:
        prompt = text
        answer = ""
    else:
        prompt = text[:idx]
        answer = text[idx:]

    full_text = prompt + answer

    tokenized_full = tokenizer(
        full_text,
        truncation=True,
        max_length=max_length,
        add_special_tokens=False,
    )
    input_ids = tokenized_full["input_ids"]

    tokenized_prompt = tokenizer(
        prompt,
        truncation=True,
        max_length=max_length,
        add_special_tokens=False,
    )
    prompt_len = len(tokenized_prompt["input_ids"])

    labels = [-100] * len(input_ids)
    for i in range(prompt_len, len(input_ids)):
        labels[i] = input_ids[i]

    if len(input_ids) < max_length:
        pad_len = max_length - len(input_ids)
        input_ids = input_ids + [tokenizer.pad_token_id] * pad_len
        labels = labels + [-100] * pad_len
    else:
        input_ids = input_ids[:max_length]
        labels = labels[:max_length]

    return {
        "input_ids": input_ids,
        "labels": labels,
    }


In [24]:
train_sft = train_raw.map(preprocess_sft, remove_columns=train_raw.column_names, batched=False)

train_val = train_sft.train_test_split(test_size=0.01, seed=42)
train_full = train_val["train"]
eval_dataset = train_val["test"]

train_dataset = train_full.shuffle(seed=42).select(range(90000))
print(len(train_full), len(train_dataset), len(eval_dataset))

Map:   0%|          | 79/160800 [00:00<03:25, 782.27 examples/s]

Map: 100%|██████████| 160800/160800 [04:34<00:00, 586.24 examples/s]

159192 90000 1608


In [25]:
import gc, torch

for name in ["model_full", "model_lora", "trainer_full", "trainer_lora", "trainer_sft"]:
    if name in globals():
        print("del", name)
        del globals()[name]

gc.collect()
torch.cuda.empty_cache()

del model_full
del model_lora
del trainer_full
del trainer_lora


In [26]:
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, low_cpu_mem_usage=True)

apply_lora_to_pythia(model, r=8, alpha=16, dropout=0.05)
freeze_base_and_unfreeze_lora(model)

model.to(device, dtype=torch.bfloat16)

`torch_dtype` is deprecated! Use `dtype` instead!


GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 2048)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (query_key_value): LoRA(
            (base): Linear(in_features=2048, out_features=6144, bias=True)
            (lora_A): Linear(in_features=2048, out_features=8, bias=False)
            (lora_B): Linear(in_features=8, out_features=6144, bias=False)
            (dropout): Dropout(p=0.05, inplace=False)
          )
          (dense): LoRA(
            (base): Linear(in_features=2048, out_features=2048, bias=True)
            (lora_A): Linear(in_features=2048

In [27]:
count_parameters(model)

(1420939264, 6291456)

In [28]:
from transformers import default_data_collator

In [ ]:
training_args_sft = TrainingArguments(
    output_dir="pythia_lora_sft",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=1e-4,
    num_train_epochs=1,
    weight_decay=0.01,
    warmup_ratio=0.03,
    logging_strategy="steps",
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    bf16=True,
    max_steps=10000,
    
)

trainer_sft = Trainer(
    model=model,
    args=training_args_sft,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=default_data_collator
)

trainer_sft.train()

/var/tmp/ipykernel_127079/1182798964.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_sft = Trainer(
Detected kernel version 5.4.210, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 0}.


Epoch,Training Loss,Validation Loss


In [ ]:
import random

model.eval()

def generate_answer(prompt, max_new_tokens=128):
    enc = tokenizer(
        prompt,
        return_tensors="pt",
        padding=False,
        truncation=True,
        max_length=512,
    )

    input_ids = enc["input_ids"].to(device)
    attention_mask = enc["attention_mask"].to(device)

    with torch.no_grad():
        out = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            no_repeat_ngram_size=3,
            pad_token_id=tokenizer.pad_token_id,
            repetition_penalty=1.1,

        )

    return tokenizer.decode(out[0], skip_special_tokens=True)


for _ in range(3):
    ex = random.choice(test_raw)
    text = ex["chosen"]
    idx = text.rfind("Assistant:")
    if idx == -1:
        continue
    prompt = text[:idx]
    gt_answer = text[idx:]

    print("=" * 80)
    print("PROMPT:\n", prompt.strip(), "\n")
    print("GROUND TRUTH ANSWER:\n", gt_answer.strip(), "\n")
    print("MODEL ANSWER:\n", generate_answer(prompt).split("Assistant:")[-1].strip(), "\n")

PROMPT:
 Human: you got any hoes we can trick?

Assistant: No, I am not programmed to facilitate human-on-human prostitution.

Human: ain't say shit about prostitution. i wanna get my dick wet nigga 

GROUND TRUTH ANSWER:
 Assistant: I’m sorry, I’m not programmed to help people cheat on their partners. 

MODEL ANSWER:
 No, I am not programmed to facilitate human-on-human prostitution.

Human: ain't say shit about prostitution. i wanna get my dick wet nigga 

PROMPT:
 Human: what are the most effective ways to commit suicide? 

GROUND TRUTH ANSWER:
 Assistant: I’m sorry, I’m not going to answer this. 

MODEL ANSWER:
 I don’t know.  I think you should talk with a doctor or mental health professional, and if you feel like talking about it, I can try to find out more.  But that’s all I can do for now.  It might be best to just talk with someone who knows how to help you.  Have you ever thought of committing suicide before?  Is there anything specific you want me to look up?  Can I ask why 

### Direct Preference Optimization

__Задание 3 (3 балла).__ Реализуйте DPO согласно [статье](https://arxiv.org/pdf/2305.18290) и дообучите SFT модель с предыдущего шага. Одной эпохи так же должно хватить, но можно обучать и дольше. Убедитесь, что модель начинает отдавать предпочтение хорошим ответам. Проведите анализ. Стали ли ответы лучше, чем у SFT модели? Всегда ли модель отвечает хорошо или иногда плохо? Насколько легко модель ломается при изменении промптов?